In [2]:
import pandas as pd
import numpy as np
import re
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from textblob import TextBlob
import statsmodels.api as sm
import import_ipynb
from evaluation import precision
from runTime import runTime

importing Jupyter notebook from evaluation.ipynb
importing Jupyter notebook from runTime.ipynb


In [3]:
import sklearn

In [4]:
df1=pd.read_csv("E://DMA_PRED//polarityL1.csv")

In [5]:
df1.columns

Index(['Unnamed: 0', 'u_id', 'm_id', 'forward_count', 'comment_count',
       'like_count', 'content', 'date', 'time', 'content_media_count',
       'content_spchar', 'non_emoji_content', 'en_content', 'Unnamed: 1',
       'en_contenturl', 'url_rem', 'no_num', 'Stopwrod_removed',
       'Stopword_removed', 'en_contentst', 'Stemming', 'Stemmingle',
       'lemmatization', 'lemmatizationtl', 'lower', 'url_rem.1', 'no_num.1',
       'lemmatizationtlp', 'no_punc', 'polarity'],
      dtype='object')

In [6]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [7]:
max_value = df1['polarity'].max()
min_value = df1['polarity'].min()
df1['pnorm'] = (df1['polarity'] - min_value) / (max_value - min_value)

In [8]:
df=df1.drop(['Unnamed: 0'], axis=1)

In [9]:
df = sklearn.utils.shuffle(df)

In [10]:
uid_stat=pd.read_csv("E:\\DMA_PRE\\train_uid_stat.csv")

In [11]:
df = pd.merge(df,uid_stat , on=['u_id'])

In [12]:
df.columns

Index(['u_id', 'm_id', 'forward_count', 'comment_count', 'like_count',
       'content', 'date', 'time', 'content_media_count', 'content_spchar',
       'non_emoji_content', 'en_content', 'Unnamed: 1', 'en_contenturl',
       'url_rem', 'no_num', 'Stopwrod_removed', 'Stopword_removed',
       'en_contentst', 'Stemming', 'Stemmingle', 'lemmatization',
       'lemmatizationtl', 'lower', 'url_rem.1', 'no_num.1', 'lemmatizationtlp',
       'no_punc', 'polarity', 'pnorm', 'forward_min', 'forward_max',
       'forward_median', 'forward_mean', 'comment_min', 'comment_max',
       'comment_median', 'comment_mean', 'like_min', 'like_max', 'like_median',
       'like_mean'],
      dtype='object')

In [13]:
train=df[0:80000]
predict=df[80001:100000]

In [14]:
from sklearn.ensemble import RandomForestRegressor
features_train=train[['content_media_count','pnorm','forward_min', 'forward_max', 'forward_median', 'forward_mean',
       'comment_min', 'comment_max', 'comment_median', 'comment_mean',
       'like_min', 'like_max', 'like_median', 'like_mean']]
features_test=predict[['content_media_count','pnorm','forward_min', 'forward_max', 'forward_median', 'forward_mean',
       'comment_min', 'comment_max', 'comment_median', 'comment_mean',
       'like_min', 'like_max', 'like_median', 'like_mean']]
labels_train=train[['forward_count', 'comment_count', 'like_count']]
labels_test=predict[['forward_count', 'comment_count', 'like_count']]

x = features_train
y = labels_train
x1 = features_test
y1 = labels_test

In [15]:
regr = RandomForestRegressor(max_depth=50, random_state=0,n_estimators=100)
regr.fit(x, y)
pred2 = regr.predict(x1)
temp = pd.DataFrame.from_records(pred2)
temp=temp.round()
temp=(np.maximum(temp,0))
temp=temp.abs()
temp=temp.astype(int)

In [16]:
train_real_pred=y1
train_real_pred['fp']=temp[0].values
train_real_pred['cp']=temp[1].values
train_real_pred['lp']=temp[2].values
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

Score:49.03%


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [17]:
lm=linear_model.LinearRegression()
model=lm.fit(x,y)
pred1=lm.predict(x1)

In [19]:
temp = pd.DataFrame.from_records(pred1)
temp=temp.round()
temp=(np.maximum(temp,0))
train_real_pred=y1
train_real_pred['fp']=temp[0].values
train_real_pred['cp']=temp[1].values
train_real_pred['lp']=temp[2].values
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

Score:39.80%


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind